# Układy równań liniowych
# oraz różne sposoby ich rozwiązywania

## Przykładowy układ równań i sposób jego przedstawienia za pomocą macierzy
Taki układd równań:
$$
    \begin{cases}
        2.3x_1 - 3.4x_2 + 0.66x_3 = 13.1 \\ \\
        -0.23x_1 + 6.2x_2 + 12.2x_3 = 2.0 \\ \\
        22.12x_1 + 3.33x_2 - 7.89x_3 = -23.12
    \end{cases}
$$
można zapisać za pomocą macierzy w ten sposób:
$$
    \begin{gather}
        \begin{bmatrix}
            2.3 & -3.4 & 0.66 \\
            -0.23 & 6.2 & 12.2 \\
            22.12 & 3.33 & -7.89
        \end{bmatrix}
        *
        \begin{bmatrix}
            x_1 \\
            x_2 \\
            x_3
        \end{bmatrix}
        =
        \begin{bmatrix}
            13.1 \\
            2.0 \\
            -23.12
        \end{bmatrix}
    \end{gather}
$$
gdzie kolejno $ \bold{A} $ to macierz podstawowa (główna) układu, $ \bold{x} $ to wektor niewiadomych oraz $ \bold{b} $ to wektor wyrazów wolnych. Ogólnie równanie macierzowe przedstawiamy więc, w następujący sposób:

$$ \bold{A} * \bold{x} = \bold{b} $$

Spotkać się też można z macierzą zawierającą wyrazy wolne, należy to traktować jak wiele układów równań, gdzie każdy składa się z takich samych współczynników oraz dla każdego układu przypisujemy kolejne kolumny macierzy wyrazów wolnych.

## Analiza sposobów rozwiązywania układów macierzowych

### Dane testowe
Jako dane testowe przyjmujemy macierz pasmową. Macierz pasmowa (wstęgowa) to kwadratowa macierz rzadka, której wszystkie elementy są zerowe poza diagonalą i pasmem (wstęgą) wokół niej. Mając daną macierz $ {\displaystyle n\times n,}$ jej elementy $ {\displaystyle a_{i,j}} $ są niezerowe, gdy ${\displaystyle i-k_{1}\leqslant j\leqslant i+k_{2},}$ gdzie ${\displaystyle k_{1,2}\geqslant 0}$ określają tzw. szerokość pasma. Jednak aby uprościć obliczenia będziemy operować na macierzach w formacie pełnym.

Nasza macierz główna będzie wymiaru $108\times 108, k_1 = k_2 = 2$ oraz będzie zawierać następujące elementy:

$$
\[
  \bold{A} =
  \left[ {\begin{array}{cccc}
    14 & -1 & -1 & 0 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & 14 & -1 & -1 & 0 & 0 & 0 & \cdots & 0 \\
    -1 & -1 & 14 & -1 & -1 & 0 & 0 & \cdots & 0 \\
    0 & -1 & -1 & 14 & -1 & -1 & 0 & \cdots & 0 \\
    \vdots &  & \ddots & \ddots & \ddots & \ddots & \ddots &  & \vdots \\
    0 & \cdots & 0 & -1 & -1 & 14 & -1 & -1 & 0 \\
    0 & \cdots & 0 & 0 & -1 & -1 & 14 & -1 & -1 \\
    0 & \cdots & 0 & 0 & 0 & -1 & -1 & 14 & -1 \\
    0 & \cdots & 0 & 0 & 0 & 0 & -1 & -1 & 14 \\
  \end{array} } \right]
\]
$$

wektor wyrazów wolnych:

$$
\[
  \bold{b} =
  \left[ {\begin{array}{cccc}
    sin(5) \\
    sin(10) \\
    sin(15) \\
    \vdots \\
    sin(530) \\
    sin(535) \\
    sin(540)
  \end{array} } \right]
\]
$$


In [8]:
from math import sin
from matrix import Matrix

def generate_test_data(size: int, a_1: float, a_2: float, a_3: float, f_4: int):
    # print("N =", size, "\na1 =", a_1, "\na2 =", a_2, "\na3 =", a_3)

    a =  Matrix(size)
    rows, cols = a.get_size()
    for i in range(rows):
        for j in range(cols):
            if i == j:
                a.set_at(i, j, a_1)

            else:
                distance = abs(i - j)
                if distance == 1:
                    a.set_at(i, j, a_2)

                if distance == 2:
                    a.set_at(i, j, a_3)

    b = Matrix(n, m=1)
    for i in range(b.get_size()[0]):
        b.set_at(i, value=sin(i * (f_4 + 1)))

    return a, b

my_index = "184934"
n = 9 * int(my_index[len(my_index) - 2]) * int(my_index[len(my_index) - 1])
a1 = 5 + float(my_index[3])
a2 = a3 = -1.0
f = int(my_index[2])
matrix_a, vector_b = generate_test_data(n, a1, a2, a3, f)

print(matrix_a.head(1), vector_b.head(3))

[[14.0, -1.0, -1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] [[0.0], [-0.9589242746631385], [-0.5440211108893698]]


### Metody iteracyjne
Jako pierwsze testować będziemy metody iteracyjne, a konkretnie metodę Jacobiego oraz metodą Gaussa-Seidla. Metody iteracyjne to metody, które nie uzyskują dokładnego rozwiązania, a tylko jego przybliżenie, w zamian za szybkość działania.

Bardzo ważne dla algorytmów iteracyjnych jest określenie, w jakim momencie osiągneliśmy satysfakcjonujące przybliżenie rozwiązania. W tym celu skorzystamy z tzw. *wektora residuum*, który dla *k*-tej iteracji przyjmuje postać:

$$
    \bold{res^{(k)}} = \bold{A}\bold{x}^{(k)} - \bold{b}
$$

Wielkość błędu natomiast jesteśmy w stanie zbadać określając jego normę euklidesową.

$$
    ||\bold{e}||_2 = \sqrt{\sum_{j=1}^{n}e_j^2}
$$

In [3]:
from math import sqrt

def euclidean_norm(vec: Matrix):
    return sqrt(sum([vec.get_at(i) ** 2 for i in range(vec.get_size()[0])]))

print(euclidean_norm(vector_b))

7.341106924346289


#### Metoda Jacobiego
Metoda Jacobiego to jedna z metod iteracyjnych. Metoda ta służy do rozwiązywania układów macierzowych **diagonalnie dominujących**, to takich, dla których suma modułów wartości elementów w danym rzędzie, poza elementem na diagonali, jest mniejsza od modułu wartości na diagonali.

$$
    |a_{ii}| > \sum_{}^{}_{i\ne{j}}|a_{ij}|
$$

Metoda ta wykorzystuje szybkie obliczeniowo rozwiązywanie układów równań zapisanych za pomocą macierzy **diagonalnych**.
Pierwszym krokiem jest rozbicie macierzy systemowej na trzy macierze: $ \bold{A} = \bold{L} + \bold{U} + \bold{D} $, gdzie kolejno macierz $ \bold{L} $ to macierz trójkątna dolna, zawierająca elementy **poniżej** głównej diagonali macierzy $ \bold{A} $, macierz $ \bold{L} $ to macierz trójkątna górna, zawierająca elementy **powyżej** głównej diagonali macierzy $ \bold{A} $, macierz $ \bold{D} $ to macierz diagonalna, zawierająca elementy z głównej diagonali macierzy $ \bold{A} $.



In [5]:
from matrix import diagonal_solver

def jacobi(a: Matrix, b: Matrix, epsilon: float):
    r = Matrix(a.get_size()[0], m=1, value=1.0)
    it = 0

    l_sum_u = a.tri_l(1) + a.tri_u(1)
    d = a.diag()

    const_1 = diagonal_solver(-d, l_sum_u) # TODO comment
    const_2 = diagonal_solver(d, b) # TODO comment

    while True:
        r = const_1 * r + const_2

        residuum = a * r - b
        if euclidean_norm(residuum) <= epsilon:
            break

        it += 1
    return it, r

eps = 1e-9
iterations, result = jacobi(matrix_a, vector_b, eps)
print(iterations, result)

20 [[-0.007541429934607744], [-0.0686305618789963], [-0.03694945721119558], [0.042587295493920534], [0.06031340623917146], [-0.00879909040787026], [-0.06539631127411015], [-0.02833990609442667], [0.04930885649272106], [0.05631052933537321], [-0.01736348157815832], [-0.06616158269077792], [-0.0201716915607347], [0.05471765945700439], [0.05121434402873894], [-0.02566251690507197], [-0.0657733161830597], [-0.011652288600959375], [0.059162688800857614], [0.04521672378526106], [-0.03351013942709633], [-0.06422784255266173], [-0.0029278809404998463], [0.06256678434506487], [0.03842354251525188], [-0.04076817225348106], [-0.061552320187408784], [0.0058480409294720415], [0.06487005633425183], [0.03095432297769975], [-0.04730891451809502], [-0.05779382314051216], [0.014521070166848492], [0.06603198014347755], [0.022940481434457132], [-0.053017285939544995], [-0.05301847982295814], [0.022938610231887663], [0.06603211244807011], [0.014523016429037701], [-0.05779285128313255], [-0.0473103094219076

### Metoda Gaussa-Seidla


In [ ]:
def gauss_seidl(a: Matrix, b: Matrix, epsilon: float):
    pass

### Metody bezpośrednie

## Źródła
    1. https://en.wikipedia.org/wiki/Band_matrix